In [4]:
from google.cloud import videointelligence as vi
import time
from typing import Optional, Sequence
from datetime import timedelta
from google.oauth2 import service_account
import argparse

In [5]:
video_uri='gs://live2eat-bootcamp/Dish Videos/Bak Chor Mee.mp4'

In [3]:
# credentials = service_account.Credentials.from_service_account_file(
#     '/home/shyann/code/seshyblack/live2eat/notebooks/lewagon-livetoeat-cdabf6bc01ed.json')

In [6]:
import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS']='/home/shyann/code/seshyblack/live2eat/notebooks/lewagon-livetoeat-d53331dc5e74.json'

In [7]:
client = vi.VideoIntelligenceServiceClient()
job = client.annotate_video(
    input_uri=video_uri,
    features=['LABEL_DETECTION'],
)
result = job.result()
print(result)

annotation_results {
  input_uri: "/live2eat-bootcamp/Dish Videos/Bak Chor Mee.mp4"
  segment_label_annotations {
    entity {
      entity_id: "/m/01mtb"
      description: "cooking"
      language_code: "en-US"
    }
    category_entities {
      entity_id: "/m/018tkd"
      description: "service"
      language_code: "en-US"
    }
    segments {
      segment {
        start_time_offset {
        }
        end_time_offset {
          seconds: 113
          nanos: 900000000
        }
      }
      confidence: 0.6674160957336426
    }
  }
  segment_label_annotations {
    entity {
      entity_id: "/m/02q08p0"
      description: "dish"
      language_code: "en-US"
    }
    category_entities {
      entity_id: "/m/02wbm"
      description: "food"
      language_code: "en-US"
    }
    segments {
      segment {
        start_time_offset {
        }
        end_time_offset {
          seconds: 113
          nanos: 900000000
        }
      }
      confidence: 0.8671702742576599
    }
 

In [8]:
def track_objects(
    video_uri: str,
    segments: Optional[Sequence[vi.VideoSegment]] = None
) -> vi.VideoAnnotationResults:

    video_client = vi.VideoIntelligenceServiceClient()
    features = [vi.Feature.OBJECT_TRACKING]
    context = vi.VideoContext(segments=segments)
    request = vi.AnnotateVideoRequest(
        input_uri=video_uri,
        features=features,
        video_context=context,
    )
    print(f'Processing video "{video_uri}"...')

    operation = video_client.annotate_video(request)
    return operation.result().annotation_results[0]


In [9]:
segment = vi.VideoSegment(
    start_time_offset=timedelta(seconds=0),
    end_time_offset=timedelta(seconds=240),
    )

In [10]:
results = track_objects(video_uri,[segment])

Processing video "gs://live2eat-bootcamp/Dish Videos/Bak Chor Mee.mp4"...


In [11]:
def print_object_frames(
    results: vi.VideoAnnotationResults, entity_id: str, min_confidence: float = 0.7
):
    def keep_annotation(annotation: vi.ObjectTrackingAnnotation) -> bool:
        return all(
            [
                annotation.entity.entity_id == entity_id,
                min_confidence <= annotation.confidence,
            ]
        )

    annotations = results.object_annotations
    annotations = [a for a in annotations if keep_annotation(a)]
    for annotation in annotations:
        description = annotation.entity.description
        confidence = annotation.confidence
        print(
            f" {description},"
            f" confidence: {confidence:.0%},"
            f" frames: {len(annotation.frames)} ".center(80, "-")
        )
        for frame in annotation.frames:
            t = frame.time_offset.total_seconds()
            box = frame.normalized_bounding_box
            print(
                f"{t:>7.3f}",
                f"({box.left:.5f}, {box.top:.5f})",
                f"({box.right:.5f}, {box.bottom:.5f})",
                sep=" | ",
            )


In [12]:
food_entity_id = '/m/02wbm'
print_object_frames(results, food_entity_id)

---------------------- food, confidence: 71%, frames: 90 -----------------------
 45.200 | (0.58392, 0.80447) | (0.98554, 0.98635)
 45.300 | (0.53413, 0.54712) | (0.98995, 0.98322)
 45.400 | (0.52188, 0.37875) | (0.99672, 0.98623)
 45.500 | (0.51105, 0.24400) | (0.99981, 0.98756)
 45.600 | (0.51564, 0.15552) | (0.99712, 0.97299)
 45.700 | (0.51980, 0.12437) | (0.99706, 0.95529)
 45.800 | (0.53058, 0.13037) | (0.99613, 0.94855)
 45.900 | (0.53657, 0.14490) | (0.99830, 0.95592)
 46.000 | (0.53961, 0.16014) | (1.00017, 0.96687)
 46.100 | (0.53938, 0.17055) | (1.00157, 0.97773)
 46.200 | (0.54078, 0.17924) | (1.00092, 0.98435)
 46.300 | (0.54115, 0.18324) | (1.00059, 0.98882)
 46.400 | (0.54217, 0.18532) | (0.99987, 0.99035)
 46.500 | (0.54123, 0.18650) | (0.99977, 0.99139)
 46.600 | (0.54790, 0.19187) | (0.99712, 0.98809)
 46.700 | (0.53356, 0.17966) | (1.00304, 0.99435)
 46.800 | (0.53110, 0.17514) | (1.00098, 0.99414)
 46.900 | (0.56320, 0.20743) | (0.98989, 0.98531)
 47.000 | (0.57858,